In [0]:
import psycopg2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")
#import plotly
#import plotly.graph_objs as go

#import plotly.plotly as py
#import cufflinks as cf


#from sklearn.metrics import r2_score

#import rpy2
#print(rpy2.__version__)#

#import warnings
#from collections import Counter

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [0]:
print(tf.test.gpu_device_name()) 

!cat /proc/meminfo


/device:GPU:0
MemTotal:       13335276 kB
MemFree:        10211444 kB
MemAvailable:   12077272 kB
Buffers:           67000 kB
Cached:          1946156 kB
SwapCached:            0 kB
Active:           975168 kB
Inactive:        1757112 kB
Active(anon):     678096 kB
Inactive(anon):     4448 kB
Active(file):     297072 kB
Inactive(file):  1752664 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               936 kB
Writeback:             0 kB
AnonPages:        719020 kB
Mapped:           501188 kB
Shmem:              5000 kB
Slab:             154016 kB
SReclaimable:     114420 kB
SUnreclaim:        39596 kB
KernelStack:        4068 kB
PageTables:         6948 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6667636 kB
Committed_AS:    2310556 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
Shm

In [0]:

try:
    connection = psycopg2.connect(user="sesyr",
                                  password="SeSyR54",
                                  host="51.254.68.36",
                                  port="5432",
                  
                                  database="donnees_trafic")
    cursor = connection.cursor()

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")
    
    postgreSQL_select_Query = """
    WITH metadata AS (
          SELECT ist.macrosection,
         ist.code_centrale,
         cnl.code_canal,
         sc.sens,
         cc.id_nature_mesure, cc.libelle
        FROM rdv.info_section_troncon ist
        JOIN rdv.sens_canal sc ON sc.sens::text = ist.sens::text
    JOIN myrabel.centrale c ON c.code_centrale = ist.code_centrale::bpchar
    JOIN myrabel.canal cnl ON cnl.id_centrale = c.id_centrale AND cnl.code_canal::text = sc.canal::text
    JOIN myrabel.cal_canal cc ON cc.id_canal = cnl.id_canal AND cc.libelle ~~ '%QT_360 %'::text
    
    ), metadatavt AS (
          SELECT distinct ist.macrosection,
         ist.code_centrale,
         cnl.code_canal,
         sc.sens,
         cc.id_nature_mesure, cc.libelle
        FROM rdv.info_section_troncon ist
        JOIN rdv.sens_canal sc ON sc.sens::text = ist.sens::text
    JOIN myrabel.centrale c ON c.code_centrale = ist.code_centrale::bpchar
    JOIN myrabel.canal cnl ON cnl.id_centrale = c.id_centrale AND cnl.code_canal::text = sc.canal::text
    JOIN myrabel.cal_canal cc ON cc.id_canal = cnl.id_canal AND cc.libelle ~~ '%VT_360 %'::text
    )
    
    SELECT 
       
       m.macrosection,
       m.sens,
       m.code_centrale,
       m.code_canal, 
       
        
       ts,
       dq.valeur AS qt,
       dv.valeur AS vt
       
       
      FROM metadata  m
      JOIN metadatavt v
      ON m.macrosection = v.macrosection AND m.sens = v.sens AND m.code_centrale = v.code_centrale AND  m.code_canal=  v.code_canal
      
      CROSS JOIN generate_series('2017-05-01' , '2017-07-01','00:06:00'::interval) AS  ts(ts)
      
      LEFT JOIN myrabel.mesure_2 dq    
      ON dq.id_nature_mesure = m.id_nature_mesure
      AND dq.date_mesure = ts 
      
        LEFT JOIN myrabel.mesure_2 dv    
      ON dv.id_nature_mesure = v.id_nature_mesure
      AND dv.date_mesure = ts
      
      
        --WHERE(  dv.date_mesure IS NULL OR dq.date_mesure IS NULL OR dv.date_mesure = dq.date_mesure )
      
     -- WHERE dq.date_mesure IS NOT  NULL AND dv.date_mesure IS NULL  --OR dq.date_mesure IS NULL
      
      """
    

    cursor.execute(postgreSQL_select_Query)
    print("Selecting rows from mobile table using cursor.fetchall")
    records = cursor.fetchall() 
   
   # print("Print each row and it's columns values")
   #for row in records:
       #   print("Macrosection = ", row[0])
        #  print("sens = ", row[1])
         # print("code centrale   = ", row[2])
          #print(" code canal = ", row[3])
        #  print("date_mesure = ", row[4])
         # print("qt_360 = ", row[5]) 
          #print("Vt_360 = ", row[6],"\n" )
            
            
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
        
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")            

You are connected to -  ('PostgreSQL 10.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-28), 64-bit',) 

Selecting rows from mobile table using cursor.fetchall
PostgreSQL connection is closed


In [0]:
DataFrameDict=df.groupby(['code_canal','code_centrale'])
    
#print 
for key, item in DataFrameDict:
    print(DataFrameDict.get_group(key).count(), "\n\n")
    #DataFrameDict.get_group(key).to_csv(r'C:\Users\hp\Desktop\Stage IOT TS\devellopement\grouping_by_codes'+ str(key) +'.csv')
    


In [0]:
df = pd.DataFrame(records, columns=['Macrosection','sens','code_centrale','code_canal','date','qt_360','vt_360'])

#strip code canal which is a char of 3 character with whitespace

#df['code_canal'] = df['code_canal'].str.strip()
df = df.drop(["sens", "Macrosection"], axis=1)


df = df.replace(0,np.NaN) # np.NaN ca va marcher , np.nan doesnt work anymore !!

df.head()

In [0]:
#store data in alias dictionary
DataGp = {elem : pd.DataFrame for elem,it in DataFrameDict}

for key in DataGp.keys():
        print(key,end ="  ")
        DataGp[key] = DataFrameDict.get_group(key)


del DataGp['1  ', 'SITE_08']


('0  ', 'MZE54.3')  ('0  ', 'MZE54.H')  ('0  ', 'SITE_03')  ('0  ', 'SITE_05')  ('0  ', 'SITE_08')  ('1  ', 'MZE54.3')  ('1  ', 'MZE54.4')  ('1  ', 'MZE54.H')  ('1  ', 'MZE54.K')  ('1  ', 'SITE_01')  ('1  ', 'SITE_03')  ('1  ', 'SITE_05')  ('1  ', 'SITE_08')  ('1  ', 'SITE_09')  ('2  ', 'MZE54.H')  ('2  ', 'SITE_03')  ('2  ', 'SITE_05')  ('2  ', 'SITE_08')  ('3  ', 'MZE54.H')  ('3  ', 'SITE_01')  ('3  ', 'SITE_03')  ('3  ', 'SITE_05')  ('3  ', 'SITE_08')  ('3  ', 'SITE_09')  ('5  ', 'SITE_01')  ('5  ', 'SITE_08')  ('5  ', 'SITE_09')  

In [0]:
from IPython.display import display

pd.options.display.max_columns = None
#numerical attributes:
display(DataFrameDict.describe())

qt_360                                            \
                            count        mean         std  min   25%    50%   
code_canal code_centrale                                                      
0          MZE54.3        14640.0  136.642760  100.778471  2.0  39.0  131.0   
           MZE54.H        14622.0  135.276912  103.180416  2.0  36.0  128.0   
           SITE_03        14328.0   76.711753   41.473932  2.0  34.0   88.0   
           SITE_05        14468.0   84.036978   49.020774  1.0  34.0   93.0   
           SITE_08        14466.0   87.267109   52.024999  1.0  33.0   97.0   
1          MZE54.3        14639.0  141.254252  104.099162  1.0  39.0  139.0   
           MZE54.4        14633.0  117.079136   90.240136  1.0  33.0  117.0   
           MZE54.H        14622.0  117.843729   84.675812  1.0  34.0  119.0   
           MZE54.K        14639.0  117.836259   89.838539  1.0  32.0  117.0   
           SITE_01        14298.0   37.188908   22.057392  1.0  14.0   43.0   
           SITE_03        14327.0   82.267886   46.847388  1.0  35.0   95.0   
           SITE_05        14467.0   84.072372   48.053135  1.0  35.0   98.0   
           SITE_08        14127.0   11.554329    7.607775  1.0   6.0   10.0   
           SITE_09        14447.0   24.744376   17.035452  1.0   9.0   21.0   
2          MZE54.H        11757.0    4.930084    3.693291  1.0   2.0    4.0   
           SITE_03        13630.0   62.945048   63.588926  1.0   7.0   47.0   
           SITE_05        13767.0   54.673930   54.559154  1.0   6.0   41.0   
           SITE_08        13233.0   53.442757   55.982020  1.0   5.0   39.0   
3          MZE54.H        13541.0   25.652832   28.645417  1.0   5.0   18.0   
           SITE_01        12124.0   29.143847   27.423198  1.0   5.0   24.0   
           SITE_03        13477.0   62.304370   60.428705  1.0   7.0   48.0   
           SITE_05        13538.0   59.081844   58.824836  1.0   6.0   44.0   
           SITE_08        14466.0   76.334785   51.000677  1.0  25.0   84.0   
           SITE_09        14634.0   67.610906   41.513811  1.0  27.0   75.0   
5          SITE_01        14303.0   61.406348   44.601980  1.0  19.0   56.0   
           SITE_08        14307.0   49.893758   48.075861  1.0   7.0   39.0   
           SITE_09        13658.0   56.301874   54.149038  1.0   7.0   48.0   

                                         vt_360                         \
                            75%    max    count        mean        std   
code_canal code_centrale                                                 
0          MZE54.3        207.0  446.0  14639.0   96.499556   7.725477   
           MZE54.H        206.0  468.0  14622.0   89.839215   9.245791   
           SITE_03        113.0  164.0  14328.0   95.404934   8.570058   
           SITE_05        125.0  209.0  14467.0   94.763726   9.772555   
           SITE_08        131.0  210.0  14462.0   84.936235   6.396083   
1          MZE54.3        217.0  542.0  14638.0   97.353532   7.165851   
           MZE54.4        175.0  484.0  14633.0  104.672453   7.966575   
           MZE54.H        181.0  375.0  14622.0   98.595883   6.896387   
           MZE54.K        179.0  441.0  14639.0   99.118519   6.670772   
           SITE_01         56.0  106.0  14291.0   99.756904   8.446648   
           SITE_03        121.0  197.0  14327.0   98.007268   7.951165   
           SITE_05        125.0  192.0  14467.0  101.103457   7.704175   
           SITE_08         17.0   63.0      0.0         NaN        NaN   
           SITE_09         40.0   92.0  14391.0   79.705369   7.185485   
2          MZE54.H          7.0   25.0  11748.0   58.807882   6.829581   
           SITE_03         97.0  314.0  13507.0  107.889751  10.578200   
           SITE_05         84.0  257.0  13669.0  108.741896  12.307003   
           SITE_08         81.0  273.0  13035.0   93.174812   7.627347   
3          MZE54.H         35.0  190.0  13526.0   53.260535   4.247865   
           SITE_01         4

In [0]:
# indexing time for plotting
for key in DataGp.keys():
    
    DataGp[key].date.replace('+02:00','+0000')
    DataGp[key].date = pd.to_datetime(DataGp[key].date, format='%Y-%m-%d %H:%M', utc = True)
    DataGp[key]= DataGp[key].set_index('date')

In [0]:
DataGp.values()

In [0]:
for key in DataGp.keys():
        
      ax=DataGp[key].plot(figsize =(22,10))
      ax.set_ylabel(key)

In [0]:
def _resample(df):
    """
    Resample the contents of a Pandas data-frame by first
    removing empty rows and columns, then up-sampling and
    interpolating the data for 1-minute intervals, and
    finally down-sampling to 60-minute intervals.
    """

    # Remove all empty rows and columns.
    df_res = df.dropna(axis=[0, 1], how='all')

    # Upsample so the time-series has data for every minute.
    #df_res = df_res.resample('1T')

    # Fill in missing values.
    df_res = df_res.interpolate(method='time')

    # Downsample so the time-series has data for every hour.
    df_res = df_res.resample('60T')

    # Finalize the resampling. (Is this really necessary?)
    df_res = df_res.interpolate()

    # Remove all empty rows.
    df_res = df_res.dropna(how='all')

    return df_res


In [0]:
for key in DataGp.keys():
      DataGp[key] = _resample(DataGp[key])

In [0]:
for key in DataGp.keys():
        
      ax=DataGp[key].plot(figsize =(22,10))
      ax.set_ylabel(key)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
# convert an array of values into a dataset matrix
def TensorForm(data, look_back):
    #determine number of data samples
    rows_data,cols_data = np.shape(data)
    
    #determine # of batches based on look-back size
    tot_batches = int(rows_data-look_back)+1
    
    #initialize 3D tensor
    threeD = np.zeros(((tot_batches,look_back,cols_data)))
    
    # populate 3D tensor
    for sample_num in range(tot_batches):
        for look_num in range(look_back):
            threeD[sample_num,:,:] = data[sample_num:sample_num+(look_back),:]
    
    return threeD

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import MinMaxScaler #, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
import sys
from datapreprocess import badzeros   ## local library datapreprocess.py


# fix random seed for reproducibility
np.random.seed(7)
# 
# ***
# 1) load dataset
# ***

dft = df[['O3','SO2','NO2', 'WS','TEMP','ATP', 'SR']]

last_col = np.shape(dft)[1] - 1

np_dft, a_dft = badzeros(dft)
print("\n")

print(a_dft)
rows_in,cols_in = np.shape(np_dft)

print("\n")
a = list(dft)  #makes a list of the column names in the dataframe
for i in range (len(a)):    #prints a list of the column names
    print (i, a[i])
# pick column to predict
try:
    target_col = int(input("Select the column number to predict (default = " + a[last_col] + "): "))
except ValueError:
    target_col = last_col   #choose last column as default

# choose look-ahead to predict   
try:
    lead_time =  int(input("How many hours ahead to predict (default = 1)? "))
except ValueError:
    lead_time = 1
    
#convert to floating numpy arrays
#dataset1 = df.fillna(0).values
dataset1 = np_dft.astype('float32')
#dataset1 = dataset1.astype('float32')
dataplot1 = dataset1[lead_time:,target_col]  #shift training data
dataplot1 = dataplot1.reshape(-1,1)

# normalize the dataset
try:
    process = input("Does the data need to be pre-preprocessed Y/N? (default = y) ")
except ValueError:
    process = 'y'
    
if process == 'Y' or 'y':
    scalerX = MinMaxScaler(feature_range=(0, 1))
    scalerY = MinMaxScaler(feature_range=(0, 1))
    
    dataset = scalerX.fit_transform(dataset1)
    dataplot = scalerY.fit_transform(dataplot1)
    
    print('\nData processed using MinMaxScaler')
else:
    print('\nData not processed')
    
# split into train and test sets
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

# prepare output arrays
# ***
# 2) dataplot[train_size:len(dataset)] changed because it should be dataplot len
# ***
trainY, testY = dataplot[0:train_size], dataplot[train_size:len(dataplot)]

n,p = np.shape(trainY)
if n < p:
    trainY = trainY.T
    testY = testY.T

# resize input sets
trainX1 = train[:len(trainY),]
testX1 = test[:len(testY),]
  
# get number of epochs
try:
    n_epochs = int(input("Number of epochs? (Default = 10)? "))
except ValueError:
    n_epochs = 10
    
# prepare input Tensors
try:
    look_back = int(input("Number of recurrent (look-back) units? (Default = " + str(lead_time+2) + ")? "))
except ValueError:
    look_back = lead_time+2

# mini-batch size    
n_batch = 72
    
# get final approval to compile and train
print('\nInput summary')
print('Loading data from ' + data_file_name)
print('Training on ' + a[target_col])
print('Number of input features is ' + str(cols_in))
print('Prediction horizon is ' + str(lead_time))
print('Number of training epochs is ' + str(n_epochs))
print('Number of recurrent units is ' + str(look_back))
print('Number of samples/batch is ' + str(n_batch))

### Make training/testing tensors       
trainX = TensorForm(trainX1, look_back)
testX = TensorForm(testX1, look_back)
print('Number of training samples is ' + str(len(trainX)))
print('Number of test samples is ' + str(len(testX)))

#### check to see if parameters are ok to continue
try:
    contin = input("Continue with model training? (Default = (y)? ")
except ValueError:
    contin = "y"
    
if contin == "n":
    sys.exit()

print('Building model...')
    
# ***
# 3) number of neurons / input_nodes increased for the LSTM layer
# ***
#input_nodes = 50
input_nodes = int(trainX.shape[2] * 2)

# trim target arrays to match input lengths
if len(trainX) < len(trainY):
    trainY = np.asmatrix(trainY[:len(trainX)])
    
if len(testX) < len(testY):
    testY = np.asmatrix(testY[:len(testX)])

model = Sequential()
# ***
# 3) Actual change on the LSTM layer
# ***
model.add(LSTM(input_nodes, activation='sigmoid', recurrent_activation='tanh', 
                input_shape=(trainX.shape[1], trainX.shape[2])))

# add dropout for generalization (default = 0.2)
#model.add(Dropout(0.2)) - can't use dropout with Keras 2.1.2 anymore :( 

# 1 neuron on the output layer
model.add(Dense(1))

# compiles the model
model.compile(loss='mean_squared_error', optimizer='nadam', metrics = [metrics.mae])

# ***
# 4) Increased the batch_size to 72. This improves training performance by more than 50 times
# and loses no accuracy (batch_size does not modify the final result, only how memory is handled)
# ***
#### Start the clock
start1 = time.clock()  

history = model.fit(trainX, trainY, epochs=n_epochs, batch_size=n_batch, validation_data=(testX, testY), shuffle=False)

    # stop clock
end1 = time.clock() 

if (end1-start1 > 60):
    print ("Model trained in {0:.1f} minutes".format((end1-start1)/60.))
else:
    print ("Model trained in {0:.1f} seconds".format((end1-start1)/1.))
# ***
# 5) test loss and training loss graph. It can help understand the optimal epochs size and if the model
# is overfitting or underfitting.
# ***
xhistory = len(history.history['loss'])
xlin = range(1,xhistory+1)
plt.close('all')
plt.plot(xlin,history.history['loss'],color="black", label='Train')
plt.plot(xlin,history.history['val_loss'], color = "black", linestyle = ':', label='Test')
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.xticks(np.arange(min(xlin), max(xlin)+1, (max(xlin)+1 - min(xlin))/10))
plt.legend()
plt.show()

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# invert predictions
trainPredict = scalerY.inverse_transform(trainPredict)
trainY = scalerY.inverse_transform(trainY)
testPredict = scalerY.inverse_transform(testPredict)
testY = scalerY.inverse_transform(testY)

# ***
# 6) calculate mean absolute error. Different than root mean squared error this one
# is not so "sensitive" to bigger errors (does not square) and tells "how big of an error"
# we can expect from the forecast on average"
# ***
print('Prediction horizon = '+ str(lead_time),'Look back = ' + str(look_back))
trainScore = mean_absolute_error(trainY, trainPredict)
print('Train Score: %.2f MAE' % (trainScore))
testScore = mean_absolute_error(testY, testPredict)
print('Test Score: %.2f MAE' % (testScore))

'''# calculate root mean squared error. 
# weights "larger" errors more by squaring the values when calculating
print('Prediction horizon = '+ str(lead_time),'Look back = ' + str(look_back))
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))
'''
########################
######  write results to file
try:
    file_write = input("Save ouput file (y/n)? (Default = n)? ")
except ValueError:
    file_write = "n"

if file_write == "y":
    # make timestamp for unique filname
    stamp = str(time.clock())  #add timestamp for unique name
    stamp = stamp[0:2] 

    # generate filename and remove extra periods
    filename = 'FinErr_lstm_'+ str(n_epochs) + str(lead_time) + '_' + stamp + '.xlsx'    #example output file
    
    if filename.count('.') == 2:
        filename = filename.replace(".", "",1)
    writer = pd.ExcelWriter(filename)
    pd.DataFrame(trainPredict).to_excel(writer,'Train-predict') #save prediction output
    pd.DataFrame(trainY).to_excel(writer,'obs-train') #save observed output
    pd.DataFrame(testPredict).to_excel(writer,'Test-predict') #save output training data
    pd.DataFrame(testY).to_excel(writer,'obs_test') 
    writer.save()
    print('File saved in ', filename)

'''
# plot baseline and predictions
plt.close('all')
plt.plot(testY)
plt.plot(testPredict)
plt.show()
'''

In [0]:
# def Bresample(tseries, rate='15T', short_rate='S', max_gap=None):
   
#     """ Resample (unevenly spaced) timeseries data linearly by first upsampling to a
#         high frequency (short_rate) then downsampling to the desired rate.
    

#     :param rate: rate that tseries should be resampled to 1H in my case
    
#     :param short_rate: intermediate upsampling rate; if None, smallest interval of tseries is used
    
#     :param max_gap: null intervals larger than `max_gap` are being treated as missing
#         data and not interpolated. if None, always interpolate. must be provided as pandas
#         frequency string format, e.g. '6h'
        
#     Copyright (c) 2017 WATTx GmbH
#     License: Apache License
#     """
    
    
#     # return series if empty
#     if tseries.empty:
#         return tseries

#     # check for datetime index
#    # assert isinstance(
#        # tseries.index[0], pd.tslib.Timestamp), 'Object must have a datetime-like index.'

#     # sort tseries by time
#     tseries.sort_index(inplace=True)

#     # create timedelta from frequency string
#     rate_delta = to_offset(rate).delta

#     # compute time intervals
#     diff = np.diff(tseries.index) / np.timedelta64(1, 's')

#     if max_gap is not None:
#         # identify intervals in tseries larger than max_gap
#         idx = np.where(np.greater(diff, to_offset(max_gap).delta.total_seconds()))[0]
#         start = tseries.index[idx].tolist()
#         stop = tseries.index[idx + 1].tolist()
#         # store start and stop indices of large intervals
#         big_gaps = list(zip(start, stop))

#     if short_rate is None:
#         # use minimal nonzero interval of original series as short_rate
#         short_rate = '%dS' % diff[np.nonzero(diff)].min()
#         # create timedelta from frequency string
#         short_rate_delta = to_offset(short_rate).delta
#         # if smallest interval is still larger than rate, use rate instead
#         if short_rate_delta > rate_delta:
#             short_rate = rate
#     else:
#         # convert frequency string to timedelta
#         short_rate_delta = to_offset(short_rate).delta
#         # make sure entered short_rate is smaller than rate
#         assert rate_delta >= short_rate_delta, 'short_rate must be <= rate'

#     # upsample to short_rate
#     tseries = tseries.resample(short_rate, how='mean').interpolate(method='cubic')

#     # downsample to desired rate
#     tseries = tseries.resample(rate, how='ffill')

#     # replace values in large gap itervals with NaN
#     if max_gap is not None:
#         for start, stop in big_gaps:
#             tseries[start:stop] = None

#     return tseries

In [0]:
#from pandas.tseries.frequencies import to_offset 


#for key in DataGp.keys():
     # DataGp[key] = Bresample(DataGp[key],'60T',None,None)

In [0]:
#for key in DataGp.keys():
        
     # ax=DataGp[key].plot(figsize =(22,10))
      #ax.set_ylabel(key)